<a href="https://colab.research.google.com/github/Rikupro-creator/LLM-prompt-recovery/blob/main/baseline_LLm_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'llm-prompt-recovery:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F67121%2F7806901%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T150707Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4aab946b7455838f7f37ddc72aeb6e646a27f6596585862c4c41ac85f0784ab1379e523b4962c20dc39e99cad2f8843171f036ad2dd81e87bbc41e6d7c0fd310f91fabd9a35f216fd335c27d5b929dcd374e7ecfc955244dc32b2ad658c11926e82c74854d3db9f174ecf232f607870859def8366ae491438f91214acddeafebdd7a22644c4eb28c6efbe14fcf8952bf33273d677b797ffcd9c90129f0bea952d36a55d525c41d84cd54a10bed4611a1226d892425a6e04169b074eb068cbbcdcefed4b5e9d8a3a61efcc9e3bae00c2c5ce452e8157b8595bdeda3a8e23d7b91e4d5e033aba106719636bdfb2e370f3f02d2ee524eee20325523c07ab4c3ea5a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# LLM prompt model

In [ ]:
# importing the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv('/kaggle/input/llm-prompt-recovery/train.csv')
test=pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

In [ ]:
train.head()

In [ ]:
print(train['original_text'])

In [ ]:
test.head()

In [ ]:
import spacy

# Load the English language model provided by spaCy
nlp = spacy.load("en_core_web_sm")

def get_first_sentence_keywords(paragraph):
    # Process the paragraph using spaCy
    doc = nlp(paragraph)

    # Extract the first sentence
    first_sentence = next(doc.sents, None)

    if first_sentence is not None:
        # Extract keywords from the first sentence
        keywords = []
        for token in first_sentence:
            # Consider only nouns, verbs, adjectives, and named entities
            if token.pos_ in ["NOUN", "VERB", "ADJ"] or token.ent_type_:
                keywords.append(token.text)
        return keywords
    else:
        return None

first_sentence_keywords = get_first_sentence_keywords(test['rewritten_text'][0])
print("Keywords from the first sentence:", first_sentence_keywords[0])


In [ ]:
example=pd.read_csv('/kaggle/input/llm-prompt-recovery/sample_submission.csv')

In [ ]:
# Retrieve keywords for each paragraph in test['rewritten_text']
keywords = []
for paragraph in test['rewritten_text']:
    first_sentence_keywords = get_first_sentence_keywords(paragraph)
    if first_sentence_keywords:
        keywords.append(first_sentence_keywords[0])
    else:
        keywords.append("")  # If no keywords found, append an empty string

# Generate rewritten sentences
rewrite = ['rewrite this essay but do it using the writing style of  ' + i if i else "rewrite this text" for i in keywords]


In [ ]:
rewrite

In [ ]:
submission=pd.DataFrame({
    'id':test.id,
    'rewrite_prompt':rewrite
})

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()